In [ ]:
!git clone https://github.com/zackproser/portfolio.git

In [ ]:
!pip install langchain_community langchain_pinecone langchain_openai unstructured langchainhub

In [ ]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import os
import glob

In [ ]:
loader = DirectoryLoader('portfolio', glob="**/*.mdx", show_progress=True, use_multithreading=True)

In [ ]:
docs = loader.load()

In [ ]:
docs

In [ ]:
from google.colab import userdata
# Set the API keys
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

In [ ]:
# Assuming you've already imported necessary libraries and blog_posts is populated as above

# Initialize embeddings and the vector store
embeddings = OpenAIEmbeddings()
index_name = "zack-portfolio"

# Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = text_splitter.split_documents(docs)

# Create a vector store for the documents using the specified embeddings
vectorstore = PineconeVectorStore.from_documents(split_docs, embeddings, index_name=index_name)

# Ask a query that is likely to score a hit against your corpus of text or data
# In my case, I have a blog post where I talk about "the programming bug"
query = "What is the programming bug?"
vectorstore.similarity_search(query)

In [ ]:
# Pinecone Index sanity checks
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone client
pc = Pinecone(api_key=os.environ.get('PINECONE_API_KEY'))

# Set the name of your Pinecone Index here
index_name = 'zack-portfolio'

index = pc.Index(index_name)

# This sanity check call should return stats for your Pinecone index, such as:
# {'dimension: 1536,
#  'index_fullness': 0.0,
#  'namespaces': {'', {'vector_count': 862}},
#  'total_vector_count': 862}
#
index.describe_index_stats()

In [ ]:
# Query the Pinecone index for related documents
query = "What is the programming bug?"

embeddings =  OpenAIEmbeddings()
vectorstore = PineconeVectorStore(embedding=embeddings, index_name=index_name)

vectorstore.similarity_search(query)